# Wrapper & Adapter layers (developer notes)

This notebook explains two patterns used in MolPy to keep the codebase modular and integrations maintainable:

- **Wrapper**: extends a MolPy object via *composition* (no implicit magic forwarding).
- **Adapter**: converts/synchronizes MolPy objects with external library objects (RDKit, file formats, etc.).

The goal is to keep *core structures* stable (`Atomistic`, `Frame`) while enabling domain- or backend-specific features without turning the core into a dependency soup.

## 1) Wrapper example (composition, explicit forwarding)

A wrapper holds an inner object and can optionally forward *selected* APIs. MolPy’s base `Wrapper` deliberately does **not** auto-forward everything via `__getattr__` — wrappers should make forwarding explicit so the public surface stays predictable.

In [ ]:
import molpy as mp
from molpy.core.wrapper import Wrapper

# Build a tiny structure we can wrap
inner = mp.Atomistic(name="demo")
a1 = inner.def_atom(symbol="C", xyz=[0.0, 0.0, 0.0])
a2 = inner.def_atom(symbol="O", xyz=[1.2, 0.0, 0.0])
inner.def_bond(a1, a2, order=1.0)

class LabeledAtomistic(Wrapper):
    """A minimal wrapper that adds metadata + forwards a small API."""

    __slots__ = ("label",)

    def __post_init__(self, label: str = "unnamed", **props):
        self.label = label
        return props

    # Explicit forwarding (choose what you want to expose)
    @property
    def atoms(self):
        return self.unwrap().atoms

    @property
    def bonds(self):
        return self.unwrap().bonds

wrapped = LabeledAtomistic(inner, label="fragment-A")
print("label:", wrapped.label)
print("n_atoms:", len(list(wrapped.atoms)))
print("n_bonds:", len(list(wrapped.bonds)))
print("unwrap is Atomistic?", isinstance(wrapped.unwrap(), mp.Atomistic))

## 2) Adapter example (sync/convert to an external library)

An adapter exists to *convert* or *synchronize* between MolPy and an external library’s representation.

Example: `RDKitAdapter` keeps a MolPy `Atomistic` and an RDKit `Mol` in sync. That lets you use RDKit for cheminformatics/embedding/SMILES while keeping MolPy as the editable structure layer.

In [ ]:
import molpy as mp
from molpy.external.rdkit_adapter import RDKitAdapter

try:
    from rdkit import Chem
except Exception as e:
    raise RuntimeError(
        "RDKit is required for this adapter example. Install via conda-forge: "
        "conda install -c conda-forge rdkit"
    ) from e

# Start from a MolPy structure
mol = mp.Atomistic(name="co")
c = mol.def_atom(symbol="C", xyz=[0.0, 0.0, 0.0])
o = mol.def_atom(symbol="O", xyz=[1.1, 0.0, 0.0])
mol.def_bond(c, o, order=2.0)

adapter = RDKitAdapter(internal=mol)
adapter.sync_to_external()
print("RDKit atoms/bonds:", adapter.mol.GetNumAtoms(), adapter.mol.GetNumBonds())
print("SMILES:", Chem.MolToSmiles(adapter.mol))

# Modify the MolPy side and re-sync
for atom in mol.atoms:
    atom["x"] = atom.get("x", 0.0) + 0.2
adapter.sync_to_external()
print("Re-synced SMILES (should match):", Chem.MolToSmiles(adapter.mol))

## Summary

- Use a **Wrapper** to extend MolPy objects via composition and explicit forwarding.
- Use an **Adapter** to bridge MolPy objects to external libraries (convert/sync).
- Keeping these layers separate makes core structures stable and integrations optional.